In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from scripts import datasources, models, variables
from synthpop.census_helpers import Census
%matplotlib inline

In [4]:
hh_synth = pd.read_csv('/home/data/fall_2018/households_v2.csv', index_col='household_id')
# per_synth = pd.read_csv('/home/data/fall_2018/persons_v2.csv', index_col='person_id')

In [5]:
c = Census('181ed8c64531c167176365d52ac460d969d54cf2')

In [6]:
pop = c.download_population_pums('06')

In [7]:
pop.rename(columns={
    'AGEP': 'age', 'RAC1P': 'race_id', 'NP': 'persons',
    'SPORDER': 'member_id', 'HISP': 'hispanic', 'RELP': 'relate',
    'SEX': 'sex', 'WKHP': 'hours', 'SCHL': 'edu', 'PERNP': 'earning',
    'JWTR': 'primary_commute_mode'},
    inplace=True)
pop['student'] = 0
pop.loc[pop.SCH.isin([2, 3]), 'student'] = 1
pop['work_at_home'] = 0
pop.loc[pop.primary_commute_mode == 11, 'work_at_home'] = 1
pop['worker'] = 0
pop.loc[pop.ESR.isin([1, 2, 4, 5]), 'worker'] = 1

In [8]:
for col in pop.columns:
    if col not in ['serialno', 'household_id', 'member_id',
                   'relate', 'age', 'sex', 'race_id', 'hispanic',
                   'student', 'worker', 'hours',
                   'work_at_home', 'edu', 'earning', 'primary_commute_mode']:
        del pop[col]

In [27]:
pop.head()

,serialno,member_id,age,primary_commute_mode,relate,edu,sex,hours,hispanic,earning,race_id,student,work_at_home,worker
0,2009000000014,1,64,NaN,0,22.0,1,50.0,1,404000.0,1,0,0,0
1,2009000000014,2,64,NaN,1,16.0,2,NaN,1,0.0,1,0,0,0
2,2009000000016,1,47,NaN,0,20.0,1,40.0,2,80000.0,8,0,0,1
3,2009000000016,2,36,NaN,1,9.0,2,NaN,2,0.0,8,0,0,0
4,2009000000016,3,20,1.0,2,14.0,1,40.0,2,40000.0,8,0,0,1


In [20]:
persons = pd.DataFrame(columns=[col for col in pop.columns if col != 'serialno'] + ['primary_commute_mode'])

In [21]:
persons.head()

,member_id,age,primary_commute_mode,relate,edu,sex,hours,hispanic,earning,race_id,student,work_at_home,worker,primary_commute_mode


In [102]:
# for i, hh in tqdm(hh_synth.iterrows(), total=len(hh_synth)):
#     hh_persons = pop[pop['serialno'] == hh['serialno']]
#     hh_persons.loc[:, 'household_id'] = int(hh.name)
#     persons = pd.concat((persons, hh_persons), ignore_index=True)

In [24]:
hh_synth.reset_index(inplace=True)

In [25]:
merged = pd.merge(hh_synth, pop, on='serialno')

In [26]:
merged.head()

,household_id,serialno,persons,building_type,cars,income,race_of_head,hispanic_head,age_of_head,workers,...,relate,edu,sex,hours,hispanic,earning,race_id,student,work_at_home,worker
0,0,2010000487191,1,6.0,1.0,85000.0,1,no,47,1.0,...,0,22.0,1,40.0,1,85000.0,1,0,0,1
1,670,2010000487191,1,6.0,1.0,85000.0,1,no,47,1.0,...,0,22.0,1,40.0,1,85000.0,1,0,0,1
2,740,2010000487191,1,6.0,1.0,85000.0,1,no,47,1.0,...,0,22.0,1,40.0,1,85000.0,1,0,0,1
3,975,2010000487191,1,6.0,1.0,85000.0,1,no,47,1.0,...,0,22.0,1,40.0,1,85000.0,1,0,0,1
4,977,2010000487191,1,6.0,1.0,85000.0,1,no,47,1.0,...,0,22.0,1,40.0,1,85000.0,1,0,0,1


In [100]:
len(merged)

7060806

In [38]:
persons_v3 = merged[[col for col in pop.columns if col != 'serialno'] + ['household_id']]

In [40]:
persons_v3.index.name = 'person_id'

In [41]:
persons_v3.to_csv('/home/data/fall_2018/persons_v3.csv')